### Assignment Overview
In this assignment, we work with the provided *salary data* to develop a Jupyter Notebook. The objective is to perform various tasks related to data processing, error handling, and file handling using Python.

### Assignment Description
The assignment consists of the following tasks:

- Import Data:


 - Import the provided salary data into your Jupyter Notebook.

- Create Employee Function:

 - Develop a Python function that accepts an employee's name as input and returns their details.
 - Data Processing with Dictionary:

- Process the salary data using a Python dictionary.
- Error Handling:

 - Implement error handling in your code to address potential issues gracefully.

- Export Employee Details:

 - Export an employee's details to a CSV file and save it within a zipped folder named "Employee Profile."
 - Unzip and Display Data with R:

- Use R to unzip the folder created in step 5 and display the data.

## 1. Import Data

In [37]:
# Import the required libraries
import pandas as pd
import numpy as np
import csv
import os
import zipfile

In [10]:
# Load the data as a Dataframe
salary_data = pd.read_csv("data/Total.csv")

# Inspect the first five rows
salary_data.head(5)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411,0,400184,Not Provided,567595.43,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966,245132,137811,Not Provided,538909.28,538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739,106088,16452.6,Not Provided,335279.91,335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.7,198307,Not Provided,332343.61,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134402,9737,182235,Not Provided,326373.19,326373.19,2011


In [16]:
# Inspect the shape of the data
print(salary_data.shape)

(312882, 9)


The data has a total of 312882 records, with 9 columns viz: Employee Name, Job Title, base pay, Over time, Other pay, benefits, Total pay, Total pay benefits and the Year.

### Data Cleaning

We will create a function that handles all the data cleaning for our dataset. This includes removing of duplicates, converting columns into their respective data types and trimming of white spaces.


In [17]:
def clean_salary_data(data):
    try:
        # Remove records where the Employee name is missing
        cleaned_data = data.dropna(subset=['EmployeeName'])
        
        # Trim whitespace from 'EmployeeName'
        cleaned_data['EmployeeName'] = cleaned_data['EmployeeName'].str.strip()
        
        # Convert numerical columns to appropriate types
        numeric_columns = ['BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits']
        cleaned_data[numeric_columns] = cleaned_data[numeric_columns].apply(pd.to_numeric, errors='coerce')
        
        # Fill missing values in numeric columns with 0
        cleaned_data[numeric_columns] = cleaned_data[numeric_columns].fillna(0)
        
        return cleaned_data

    except Exception as e:
        return f"The program encountered an error while cleaning the data: {str(e)}"
        

In [18]:
# Call the function to clean the salary data
clean_salary_data = clean_salary_data(salary_data)

In [19]:
# Inspect the cleaned data
print(clean_salary_data.shape)

(312882, 9)


## 2. Create Employee Function

In [20]:
# Let's create a function that takes in an employee's name and returns their details

def employee_details(employee_name):
    details = clean_salary_data[clean_salary_data['EmployeeName'].str.contains(employee_name, case=False, na=False)]
    
    # Implement a check to find if any employee details are found
    if not details.empty:
        return details
    else:
        return f"No information found for {employee_name}"

In [22]:
# Call the function to print details for one of the employees
employee_details('PATRICK GARDNER')

,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,0.0,326373.19,326373.19,2011


## 3. Process the salary data

In [30]:
#We create a function to process the salary data using a Python dictionary.

def process_employee_data(data):
    try: # Exception handler
        
        salary_data_dictionary = {} #create an empty dictionary
        
        for index, row in data.iterrows():
            
            employee_name = row['EmployeeName'].strip().upper()
            
            employee_information = {
                'JobTitle': row['JobTitle'],
                'BasePay': row['BasePay'],
                'OvertimePay': row['OvertimePay'],
                'OtherPay': row['OtherPay'],
                'Benefits': row['Benefits'],
                'TotalPay': row['TotalPay'],
                'TotalPayBenefits': row['TotalPayBenefits'],
                'Year': row['Year']
            }
        
            salary_data_dictionary[employee_name] = employee_information
        
        return salary_data_dictionary
    
    except Exception as e:
        return f"The program encountered an error while processing the data: {str(e)}"

In [31]:
# Process the data into a dictionary
salary_data_dictionary = process_employee_data(salary_data)

In [32]:
# We create a function to get the employee details from the dictionary above

def get_employee_details(employee_name, salary_data_dictionary):
    
    try:
        # Check if the employee exists in the dictionary
        
        if employee_name in salary_data_dictionary:
            return salary_data_dictionary[employee_name]
        else:
            return f"No information found for: {employee_name}"

    except Exception as e:
        return f"The program encountered an error while processing the data: {str(e)}"
    

In [36]:
# We call the function to get the details of PATRICK GARDNER

employee_details = get_employee_details('PATRICK GARDNER', salary_data_dictionary)
employee_details

{'JobTitle': 'DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)',
 'BasePay': 134401.6,
 'OvertimePay': 9737.0,
 'OtherPay': 182234.59,
 'Benefits': 'Not Provided',
 'TotalPay': 326373.19,
 'TotalPayBenefits': 326373.19,
 'Year': 2011}

## 4. Export Employee Details

We will now create a function to export an employee's details to a CSV file and save it within a zipped folder named "Employee Profile."

In [43]:
def export_employee_details(employee_name, salary_data_dictionary):
    try:
        employee_details = get_employee_details(employee_name, salary_data_dictionary)
        
        csv_file = f"{employee_name.replace(' ', '_')}_profile.csv"
        with open(csv_file, mode='w', newline='') as file:
            
            writer = csv.writer(file)
            
            # Write headers and employee details
            
            writer.writerow(employee_details.keys())
            writer.writerow(employee_details.values())
            
        print(f"Employee details exported to CSV: {csv_filename}")
        
        zip_folder_name = "Employee_Profile.zip"
        with zipfile.ZipFile(zip_folder_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
                zipf.write(csv_file)
                
        print(f"CSV file zipped into: {zip_folder_name}")
            
    else:
            print(employee_details)  # Print error message if employee not found
    
    except Exception as e:
        print(f"The program encountered an error: {str(e)}")

SyntaxError: invalid syntax (<ipython-input-43-30968411d282>, line 23)

In [ ]:
# Export Albert Pardini's data
export_employee_details_to_csv('ALBERT PARDINI', salary_data_dictionary)